<a href="https://colab.research.google.com/github/sourish-cmi/COVID-19/blob/master/LifeBack_Data_Processing_Model_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import os
import pandas as pd

from skimage.io import imsave, imread
import pylab
from tqdm import tqdm
import cv2 

from glob import glob

np.random.seed(42)



In [ ]:
pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 130kB/s 


In [ ]:
pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6083 sha256=aa8da8820bd709784b2940cfc854af14d73dcddd6197d8eb2f9c229ca2a5ca39
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [ ]:
import speech_recognition as sr
import ffmpeg

**Read dpression score**


In [ ]:
from pandas import DataFrame
data_folder_2014 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Labels_Testset/Testing/DepressionLabels/'
data_folder_2013='/content/drive/MyDrive/AlgoLab+LifeBack/AVEC_2013/Development_DepressionLabels'

data_folder=[data_folder_2014,data_folder_2013]
file_nms=[]
bdi = []
  
for d in data_folder:
  print('d = ',d)
  os.chdir(d)
  files = os.listdir()
  print(len(files))

  for i in range(len(files)):
    dum = files[i].split('_')
    file_nms.append(dum)
    u = pd.read_csv(files[i],sep=',',header=None)
    bdi.append(u.values[0][0])
  
print(bdi) 
print(type(bdi))
print(file_nms)
df1 = DataFrame(file_nms,columns=['Id','repeat_id','file_type'])
df1['BDI'] =bdi
print(df1.shape)
df1 = df1.drop(columns=['file_type'])
print(df1.shape)
df1 = df1.drop_duplicates()
print(df1.shape)
df1


d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Labels_Testset/Testing/DepressionLabels/
150
d =  /content/drive/MyDrive/AlgoLab+LifeBack/AVEC_2013/Development_DepressionLabels
57
[0, 22, 0, 15, 28, 7, 27, 0, 18, 3, 17, 24, 29, 6, 3, 21, 22, 23, 3, 7, 25, 0, 29, 20, 17, 1, 3, 30, 32, 25, 13, 2, 19, 22, 10, 22, 40, 8, 29, 1, 30, 6, 5, 18, 5, 44, 9, 10, 25, 6, 19, 18, 0, 9, 21, 12, 1, 3, 37, 32, 0, 12, 41, 11, 0, 9, 24, 5, 12, 4, 6, 43, 34, 10, 0, 6, 2, 0, 5, 0, 0, 15, 34, 41, 8, 12, 0, 3, 35, 9, 5, 33, 21, 29, 4, 14, 39, 15, 3, 19, 19, 11, 4, 15, 45, 27, 21, 31, 33, 17, 17, 0, 31, 6, 24, 23, 16, 7, 3, 26, 12, 25, 11, 14, 14, 8, 27, 13, 37, 3, 23, 5, 9, 19, 23, 21, 0, 1, 3, 7, 11, 11, 0, 0, 4, 33, 16, 9, 12, 16, 5, 6, 8, 12, 5, 9, 3, 23, 29, 6, 21, 18, 21, 24, 31, 6, 18, 0, 16, 17, 26, 0, 21, 21, 32, 32, 12, 12, 31, 25, 25, 34, 34, 39, 39, 9, 13, 40, 45, 45, 11, 29, 22, 20, 0, 0, 7, 4, 5, 5, 12, 27, 6, 0, 1, 3, 22]
<class 'list'>
[['364', '1', 'Depression.csv'], ['218', '3', 'Depr

,Id,repeat_id,BDI
0,364,1,0
1,218,3,22
2,365,1,0
3,225,2,15
4,234,2,28
...,...,...,...
149,240,2,16
150,204,1,5
193,310,3,20
195,316,2,0


In [ ]:
def audio_feature_extraction(data_folder,window_size=5):
  X=[]
  y=[]
  for d in data_folder:
    os.chdir(d)
    files = list(os.listdir())
    print('d = ',d)
    print(len(files))
    for i in range(len(files)):
      if i%5==0: 
      #  print('d = ',d)
         print('i = ',i)
      dum = files[i].split('_')
      dum1 = df1.loc[df1['Id'] == dum[0], ]
      bdi_score = int(dum1.loc[dum1['repeat_id'] == dum[1],'BDI' ])
      audio_file = os.path.join(d,files[i])
      #print(audio_file)
      signal, sr = librosa.load(audio_file)
      audio_length = signal.shape[0]/sr
      no_of_split = int(audio_length//window_size)
      signal_length_round =np.int(no_of_split*sr*window_size)
      signal_brks = np.array_split(signal[:signal_length_round], no_of_split)
      
      hop_length = 512 #number of short fast fourier transforms
      n_fft = 2048 #number of samples in each short fft
      # calculate duration hop length and window in seconds
      hop_length_duration = float(hop_length)/sr
      n_fft_duration = float(n_fft)/sr

      for j in range(no_of_split):
          stft = librosa.stft(signal_brks[j], n_fft=n_fft, hop_length=hop_length)
          spectrogram = np.abs(stft)
          mel_spectrogram = librosa.amplitude_to_db(spectrogram)
          mel_spectrogram_resize = cv2.resize(mel_spectrogram, (128, 128)) 

          #mel_spectrogram=librosa.feature.melspectrogram(signal_brks[j], n_fft=n_fft, hop_length=hop_length)

          X.append(mel_spectrogram_resize)
          y.append(bdi_score)
    
  X=np.array(X)
  y=np.array(y)

  return(X, y)

In [ ]:
data_folder1 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Training/Northwind'
data_folder2 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Training/Freeform'
data_folder3 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Development/Northwind'
data_folder4 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Development/Freeform'
data_folder_2013='/content/drive/MyDrive/AlgoLab+LifeBack/AVEC_2013/Development'

data_folder = [data_folder_2013,data_folder1,data_folder2,data_folder3,data_folder4]

X_train, y_train = audio_feature_extraction(data_folder=data_folder)
#X_train, y_train = audio_feature_extraction(data_folder=data_folder_2013)


d =  /content/drive/MyDrive/AlgoLab+LifeBack/AVEC_2013/Development
51
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45
i =  50
d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Training/Northwind
50
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45
d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Training/Freeform
50
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45
d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Development/Northwind
50
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45
d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Development/Freeform
50
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45


In [ ]:
print(X_train.shape)
print(y_train)


(11591, 128, 128)
[ 5  5  5 ... 21 21 31]


In [ ]:
print(np.min(X_train),np.max(X_train))

-68.614456 52.372864


In [ ]:
data_folder5 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Testing/Northwind'
data_folder6 = '/content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Testing/Freeform'

data_folder = [data_folder5,data_folder6]

X_test, y_test = audio_feature_extraction(data_folder=data_folder)


d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Testing/Northwind
50
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45
d =  /content/drive/My Drive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/Testing/Freeform
50
i =  0
i =  5
i =  10
i =  15
i =  20
i =  25
i =  30
i =  35
i =  40
i =  45


In [ ]:
print(X_test.shape)
print(y_test)
print(len(y_test))

(1208, 128, 128)
[15 15 15 ... 19 19 19]
1208


In [ ]:
print(np.min(X_test),np.max(X_test))

-100.0 46.5738


In [ ]:
X_train = (X_train+70)/140
X_test = (X_test+70)/140

In [ ]:
#data_folder = '/content/drive/MyDrive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/data_for_model_1_2/'
data_folder ='/content/drive/MyDrive/AlgoLab+LifeBack/Model_1_2/'
os.chdir(data_folder)
np.save(file='X_train.npy',arr=X_train)
np.save(file='y_train.npy',arr=y_train)


In [ ]:
#data_folder = '/content/drive/MyDrive/AlgoLab+LifeBack/AVEC2014_Audio/Audio/data_for_model_1_2/'
data_folder ='/content/drive/MyDrive/AlgoLab+LifeBack/Model_1_2'
os.chdir(data_folder)
np.save(file='X_test.npy',arr=X_test)
np.save(file='y_test.npy',arr=y_test)